In [1]:
# Dataset (Dropbox) :https://www.dropbox.com/s/1hz4o1y7g7q7tdi/CovidDataset-20200427T133042Z-001.zip?dl=0

In [2]:
#!wget https://www.dropbox.com/s/1hz4o1y7g7q7tdi/CovidDataset-20200427T133042Z-001.zip?dl=0

In [3]:
#!unzip CovidDataset.zip

In [4]:
TRAIN_PATH = "CovidDataset/Train"
VAL_PATH = "CovidDataset/Val"


In [5]:
#Importing Kiras & its Libraries for Neural Network 
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [6]:
# Now Bulding CNN Based Model In Keras (Building Layer By Layer)
# 3 or 4 CNN Layers Followed By Classification Layer

#Keeping the number of filters very small in the first layer
#Since 1st layer detects simple features of the Image
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(434,434,3))) # 217 + 217
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 432, 432, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 430, 430, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 215, 215, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 215, 215, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 213, 213, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 106, 106, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 106, 106, 64)      0

In [8]:
#There are total of 2.2 cror param in our model
# Now We Will Train Our Model From Scratch # Using Keras ImageGenerator Library, and make the data ready for CNN Model

 #Now Doing Some Augmentation on the Original Image (Making Data Ready for Model)
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale=1./255)


In [9]:
# Builting Train Data Generator Actual Function
train_generator = train_datagen.flow_from_directory(
    'CovidDataset/Train',
    target_size = (434, 434),
    batch_size = 32,
    class_mode = 'binary'  
)


Found 434 images belonging to 2 classes.


In [10]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [11]:
# Now the data generator is ready and now we only need to train our model

In [12]:
# Builting Validation Data Generator Actual Function

validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/Val',
    target_size = (434, 434),
    batch_size = 32,
    class_mode = 'binary'  
    
)

Found 60 images belonging to 2 classes.


In [ ]:
# Now we are calling Our Fit Generator for Learning
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=4,
    epochs = 12,
    validation_data = validation_generator,
    validation_steps=2
)

/Users/asfandyarkhan/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/12
4/4 [==============================] - 88s 21s/step - loss: 14.7811 - accuracy: 0.4896 - val_loss: 0.7333 - val_accuracy: 0.5000
Epoch 2/12
4/4 [==============================] - 55s 14s/step - loss: 1.6752 - accuracy: 0.5073 - val_loss: 0.6998 - val_accuracy: 0.5000
Epoch 3/12
4/4 [==============================] - ETA: 0s - loss: 0.7154 - accuracy: 0.5566 

In [ ]:
#Loss Is Very Less (Good Acuracy)

In [ ]:
model.save("model_adv.h5")

In [ ]:
model.evaluate(train_generator)

In [ ]:
model.evaluate(validation_generator)

In [ ]:
# Test Images

In [ ]:
model = load_model('model_adv.h5')

In [ ]:
import os

In [ ]:
train_generator.class_indices

In [ ]:
y_actual = []
y_test = []

In [ ]:
for i in os.listdir("./CovidDataset/Val/Normal/"):
  img = image.load_img("./CovidDataset/Val/Normal/"+i, target_size = (434,434))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict_classes(img)
  y_test.append(p[0,0])
  y_actual.append(1)

In [ ]:
for i in os.listdir("./CovidDataset/Val/Covid/"):
  img = image.load_img("./CovidDataset/Val/Covid/"+i, target_size = (434,434))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  p = model.predict_classes(img)
  y_test.append(p[0,0])
  y_actual.append(0)

In [ ]:
y_actual = np.array(y_actual)
y_test = np.array(y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_actual, y_test)

In [ ]:
import seaborn as sns

In [ ]:
labels = ['COVID-19', 'NORMAL']
plt.figure(figsize = (10,10))

sns.heatmap(cm, cmap = "plasma", annot=True, fmt='',xticklabels = labels,yticklabels = labels)